In [1]:
import os
import json
from haystack.document_stores.faiss import FAISSDocumentStore
path = 'models/faiss'

if not os.path.exists(path):
    os.makedirs(path)
    
document_store = FAISSDocumentStore(
    faiss_index_factory_str = 'Flat',
    sql_url = f'sqlite:///{path}/squad_dev.db',
    return_embedding = True
)

OpenAI tiktoken module is not available for Python < 3.8,Linux ARM64 and AARCH64. Falling back to GPT2TokenizerFast.


ValueError: The number of documents in the SQL database (1204) doesn't match the number of embeddings in FAISS (0). Make sure your FAISS configuration file points to the same database that you used when you saved the original index.

In [2]:
with open('data/squad/dev.json', 'r') as f:
    squad = json.load(f)
from haystack import Document

contexts = [sample['context'] for sample in squad]
contexts = list(set(contexts))


In [ ]:
contexts

In [4]:
squad_docs = [Document(content = sample) for sample in contexts]

In [6]:
document_store.delete_documents()

In [7]:
document_store.write_documents(squad_docs)

Writing Documents:   0%|          | 0/1204 [00:00<?, ?it/s]

In [10]:
from haystack.nodes import DensePassageRetriever

retriever = DensePassageRetriever(document_store = document_store,
                                  query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
                                  passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
                                  embed_title=True
                                 )
document_store.update_embeddings(retriever = retriever)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/492 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Updating Embedding:   0%|          | 0/1204 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/1216 [00:00<?, ? Docs/s]

In [11]:
retriever.retrieve('What subject is most abstract?')

[<Document: {'content': "A Turing machine is a mathematical model of a general computing machine. It is a theoretical device that manipulates symbols contained on a strip of tape. Turing machines are not intended as a practical computing technology, but rather as a thought experiment representing a computing machine—anything from an advanced supercomputer to a mathematician with a pencil and paper. It is believed that if a problem can be solved by an algorithm, there exists a Turing machine that solves the problem. Indeed, this is the statement of the Church–Turing thesis. Furthermore, it is known that everything that can be computed on other models of computation known to us today, such as a RAM machine, Conway's Game of Life, cellular automata or any programming language can be computed on a Turing machine. Since Turing machines are easy to analyze mathematically, and are believed to be as powerful as any other model of computation, the Turing machine is the most commonly used model 

In [12]:
document_store.save(f'{path}/squad_dev.faiss')

del document_store, retriever

In [19]:
document_store = FAISSDocumentStore.load(f'{path}/squad_dev.faiss', f'sqlite:///{path}/squad_dev.db')

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 1208: character maps to <undefined>